In [1]:
from __future__  import print_function  # Python 2/3 compatibility

import pkg_resources
import json
import pandas as pd
import boto3

pkg_resources.get_distribution("boto3").version

'1.20.3'

In [2]:
s3 = boto3.client('s3')

response = s3.get_object(
    Bucket='bearly-graph',
    Key='src/mpd.slice.0-999.json',
)
# Extracts body of the 
body = response['Body']
str_body = body.read()
s3_obj_content = json.loads(str_body)
playlists = s3_obj_content['playlists']

In [3]:
playlist_vertices=[]
album_vertices=[]
artist_vertices=[]
track_vertices=[]
vertex_list = []

for playlist in playlists:  
    vertex_dict = {}
    vertex_dict['~id'] = f"playlist_{playlist['pid']}"
    vertex_dict['name:String'] =  playlist['name']
    vertex_dict['modified_at:Int'] =  playlist['modified_at']
    vertex_dict['num_albums:Int'] =  int(playlist['num_albums'])
    vertex_dict['num_tracks:Int'] =  int(playlist['num_tracks'])
    vertex_dict['num_followers:Int'] =  int(playlist['num_followers'])
    vertex_dict['num_edits:Int'] =  int(playlist['num_edits'])
    vertex_dict['duration_ms:Int'] =  int(playlist['duration_ms'])
    vertex_dict['num_artists:Int'] =  int(playlist['num_artists'])
    vertex_dict['~label'] =  'playlist'
    playlist_vertices.append(vertex_dict)
    tracks = playlist['tracks']
    for track in tracks:
        vertex_dict = {}
        
        # album vertex
        vertex_dict['~id'] = track['album_uri']
        vertex_dict['album_name:String'] =  track['album_name']
        vertex_dict['~label'] =  'album'
        album_vertices.append(vertex_dict)
        
        # track vertex
        vertex_dict = {}
        
        vertex_dict['~id'] = track['track_uri']
        vertex_dict['track_name:String'] =  track['track_name']
        vertex_dict['duration_ms:Int'] =  int(track['duration_ms'])
        vertex_dict['~label'] =  'track'
        track_vertices.append(vertex_dict)
        
        # artist vertex
        vertex_dict = {}
        vertex_dict['~id'] = track["artist_uri"]
        vertex_dict['artist_name:String'] =  track['artist_name']
        vertex_dict['~label'] =  'artist'
        artist_vertices.append(vertex_dict)

In [5]:
# client_id=''
# client_secret=''

# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id,client_secret=client_secret))


67503
67503
67503
202509


In [7]:
# Enrich artist vertices
# for artist in artist_vertices:
#     results = spotify.artist(artist.get('~id'))
#     artist['genres:String[]'] = results.get('genres',[])
#     artist['followers:Int'] = results.get('followers',{}).get('total', 0)
#     artist['popularity:Int'] = results.get('popularity', 0)

In [ ]:
# Enrich album vertices
# for album in album_vertices:
#     results = spotify.album(album.get('~id'))
#     album['available_markets:String[]'] = results.get('available_markets',[])
#     album['total_tracks:Int'] = results.get('total_tracks',0)
#     album['popularity:Int'] = results.get('popularity',0)
#     album['label:String'] = results.get('label','')
#     album['genres:String[]'] = results.get('genres',[])
#     album['album_type:String'] = results.get('album_type',[])

In [ ]:
# Enrich track vertices
# for track in track_vertices:
#     results = spotify.track(track.get('~id'))
    

In [ ]:
# Load data into data frames


playlist_df = pd.DataFrame.from_dict(playlist_vertices)
track_df = pd.DataFrame.from_dict(track_vertices)
artist_df = pd.DataFrame.from_dict(artist_vertices)
album_df = pd.DataFrame.from_dict(album_vertices)


In [ ]:
# Load vertices into S3

b_string = playlist_df.to_csv(index=False).encode('utf-8')
s3.put_object(Body=b_string, Bucket='bearly-graph', Key='vertex/version=2/playlist_vertex_0.csv')
b_string = album_df.to_csv(index=False).encode('utf-8')
s3.put_object(Body=b_string, Bucket='bearly-graph', Key='vertex/version=2/album_vertex_0.csv')
b_string = artist_df.to_csv(index=False).encode('utf-8')
s3.put_object(Body=b_string, Bucket='bearly-graph', Key='vertex/version=2/artist_vertex_0.csv')
b_string = track_df.to_csv(index=False).encode('utf-8')
s3.put_object(Body=b_string, Bucket='bearly-graph', Key='vertex/version=2/track_vertex_0.csv')